In [10]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# UQpy imports
from UQpy.scientific_machine_learning.layers import BayesianLayer
from UQpy.scientific_machine_learning.neural_networks import FeedForwardNeuralNetwork
from UQpy.scientific_machine_learning.trainers import BBBTrainer

import logging

logger = logging.getLogger("UQpy")
logger.setLevel(logging.INFO)

torch.manual_seed(0);
np.random.seed(0);

In [11]:
width = 20
network = nn.Sequential(
    BayesianLayer(1, width),
    nn.ReLU(),
    BayesianLayer(width, width),
    nn.ReLU(),
    BayesianLayer(width, width),
    nn.ReLU(),
    BayesianLayer(width, width),
    nn.ReLU(),
    BayesianLayer(width, 1),
)
model = FeedForwardNeuralNetwork(network)

In [12]:
class SinusoidalDataset(Dataset):
    def __init__(self, n_samples=20, noise_std=0.05):
        self.n_samples = n_samples
        self.noise_std = noise_std
        self.x = torch.linspace(-1, 1, n_samples).reshape(-1, 1)
        self.y = torch.tensor(0.4 * np.sin(4 * self.x) + 0.5 * np.cos(12 * self.x) +
                              np.random.normal(0, self.noise_std, self.x.shape), dtype=torch.float)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, item):
        return self.x[item], self.y[item]

train_data = DataLoader(SinusoidalDataset(), batch_size=20, shuffle=True)

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
trainer = BBBTrainer(model, optimizer)

In [ ]:
print("Starting Training...", end="")
trainer.run(train_data=train_data, epochs=5000, beta=1e-6, num_samples=10)
print("done")

[INFO] - 2024-05-16 12:32:54,121 - UQpy: Scientific Machine Learning: Beginning training FeedForwardNeuralNetwork
[INFO] - 2024-05-16 12:32:54,126 - UQpy: Scientific Machine Learning: Epoch 1 / 5,000 Train Loss 0.20588482916355133 Train NLL 0.20481033623218536 Train KL 1074.488037109375
[INFO] - 2024-05-16 12:32:54,130 - UQpy: Scientific Machine Learning: Epoch 2 / 5,000 Train Loss 0.2054096758365631 Train NLL 0.20434018969535828 Train KL 1069.487548828125
[INFO] - 2024-05-16 12:32:54,135 - UQpy: Scientific Machine Learning: Epoch 3 / 5,000 Train Loss 0.20562739670276642 Train NLL 0.20456309616565704 Train KL 1064.3060302734375
[INFO] - 2024-05-16 12:32:54,141 - UQpy: Scientific Machine Learning: Epoch 4 / 5,000 Train Loss 0.20729409158229828 Train NLL 0.20623448491096497 Train KL 1059.6136474609375
[INFO] - 2024-05-16 12:32:54,146 - UQpy: Scientific Machine Learning: Epoch 5 / 5,000 Train Loss 0.20589642226696014 Train NLL 0.2048412263393402 Train KL 1055.1910400390625
[INFO] - 2024-0

Starting Training...

[INFO] - 2024-05-16 12:32:54,326 - UQpy: Scientific Machine Learning: Epoch 41 / 5,000 Train Loss 0.20428283512592316 Train NLL 0.20335426926612854 Train KL 928.5608520507812
[INFO] - 2024-05-16 12:32:54,330 - UQpy: Scientific Machine Learning: Epoch 42 / 5,000 Train Loss 0.204090416431427 Train NLL 0.2031644582748413 Train KL 925.9573974609375
[INFO] - 2024-05-16 12:32:54,336 - UQpy: Scientific Machine Learning: Epoch 43 / 5,000 Train Loss 0.20556876063346863 Train NLL 0.20464536547660828 Train KL 923.3931274414062
[INFO] - 2024-05-16 12:32:54,340 - UQpy: Scientific Machine Learning: Epoch 44 / 5,000 Train Loss 0.2054210752248764 Train NLL 0.20450016856193542 Train KL 920.907958984375
[INFO] - 2024-05-16 12:32:54,345 - UQpy: Scientific Machine Learning: Epoch 45 / 5,000 Train Loss 0.20498351752758026 Train NLL 0.20406508445739746 Train KL 918.43603515625
[INFO] - 2024-05-16 12:32:54,350 - UQpy: Scientific Machine Learning: Epoch 46 / 5,000 Train Loss 0.20538486540317535 Train NLL 0.20

In [ ]:
# Plotting Results
x = SinusoidalDataset().x
y = SinusoidalDataset().y
x_val = torch.linspace(-1, 1, 1000).view(-1,1)
y_val = (0.4 * torch.sin(4 * x_val) + 0.5 * torch.cos(12 * x_val))
model.train(False)
model.sample(False);

In [ ]:
pred_val = model(x_val)
fig, ax = plt.subplots()
ax.plot(
    x.detach().numpy(),
    y.detach().numpy(), '*',
    label="Data",
    color="tab:blue",
)
ax.plot(
    x_val.detach().numpy(),
    pred_val.detach().numpy(),
    label="Final Prediction",
    color="tab:orange",
)
ax.plot(
    x_val.detach(), y_val.detach(),
    label="Exact",
    color="black",
    linestyle="dashed",
)

ax.set_title("Predictions")
ax.set(xlabel="x", ylabel="f(x)")
ax.legend()
plt.show()

In [ ]:
train_loss = trainer.history["train_loss"].detach().numpy()
fig, ax = plt.subplots()
ax.plot(train_loss)
ax.set_title("Bayes By Backpropagation Training Loss")
ax.set(xlabel="Epoch", ylabel="Loss")

plt.show()

In [ ]:
model.sample(False)
mean = model(x_val)
model.sample(True)
n = 1000
samples = torch.zeros(len(x_val), n)
for i in range(n):
    samples[:, i] = model(x_val).squeeze()
variance = torch.var(samples, dim=1)
standard_deviation = torch.sqrt(variance)

x_plot = x.squeeze().detach().numpy()
mu = mean.squeeze().detach().numpy()
sigma = standard_deviation.squeeze().detach().numpy()
fig, ax = plt.subplots()
ax.plot(x_val, mu, label="$\mu$")
ax.plot(x_val.detach(), y_val.detach(), label="Exact", color="black", linestyle="dashed")
ax.fill_between(
    x_val.view(-1).detach(), mu - (3 * sigma), mu + (3 * sigma), label="$\mu \pm 3\sigma$,", alpha=0.3
)
ax.set_title("Bayesian Neural Network $\mu \pm 3\sigma$")
ax.set(xlabel="x", ylabel="f(x)")
ax.legend()

plt.show()